In [0]:
#mount the google drive to import the data
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#import all the library nesccary for the execution 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from textblob import Word
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from textblob import TextBlob
from nltk.tokenize import WordPunctTokenizer
nltk.download('wordnet')

import re
from bs4 import BeautifulSoup
from collections import OrderedDict
from itertools import *
from itertools import repeat, chain, islice
from collections import Counter

import sklearn
from sklearn.metrics import precision_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve,  roc_auc_score, classification_report


import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense , Dropout , Embedding, LSTM, Input, merge,  Concatenate 
from keras.models import Sequential
from keras.layers import  LSTM, Embedding, Dropout, Activation,CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, Flatten
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import  Conv1D,  Flatten, MaxPooling1D
from keras.optimizers import RMSprop
from keras import backend as k
from keras.engine.topology import Layer



import torch
from torch.utils.data import TensorDataset, DataLoader

The model are as follows


1.   Simple LSTM
2.   Bidirectional lstm (Bi-LSTM)
3.   CNN -LSTM
4.   Bi-LSTM with attention

# Data pre processing

In [0]:
#data cleaning process 1 - Faster
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces

    words = tok.tokenize(lower_case) 
    words = str(TextBlob(words).correct())
    w = Word(words)
    words = w.lemmatize()
    return (" ".join(words)).strip()


#nums = [0,1600000]
#%time
#print("Cleaning and parsing the tweets...\n")
#clean_tweet_texts = []
#for i in range(nums[0],nums[1]):
    #if( (i+1)%10000 == 0 ):
       # print ("Tweets %d of %d has been processed", i+1, nums[1] )
  


#clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
#clean_df['label'] = new_df.sentiment
#clean_df.head()
#clean_df.to_csv('/content/drive/My Drive/clean_tweet1.csv',encoding='utf-8')

In [0]:
#data cleaning process 1 - Slower

punctuation  =  '!"#$%^\'()*&-,+.:;<=>?[\\]_/{|}~'
def clean_function(text):
  text = text.lower()
  text = ''.join([c for c in text if c not in punctuation ])
  text = re.sub('[0-9]+', '', text)
  new_text = []
  text_split = text.split()
  for word in text_split:
    if(word[0] != '@') and ('http' not in word) and ( word.isdigit() == False):
      word = str(TextBlob(word).correct())
      w = Word(word)
      word = w.lemmatize()
      #word = PorterStemmer().stem(word)
      new_text.append(word)
  return new_text

def new_clean_function(text):
  new_text = []
  text = re.sub('[0-9]+', '', str(text))
  text_split = text.split()
  for word in text_split:
    word = str(TextBlob(word).correct())
    w = Word(word)
    word = w.lemmatize()
    new_text.append(word)
  return new_text

##nlpdata_train =  pd.read_csv('/content/drive/My Drive/clean_tweet1.csv',sep=',')
#twitter_train_data = nlpdata_train[[ 'label','text']].rename(columns={0:'label','text':'sentence'})
#twitter_train_data.head(10)


In [0]:
#import data from the drive
nlpdata_train =  pd.read_csv('/content/drive/My Drive/clean_tweet1.csv',sep=',')
twitter_train_data = nlpdata_train[[ 'label','text']].rename(columns={0:'label','text':'sentence'})
twitter_train_data1 = twitter_train_data.sample(10000)
twitter_train_data1 = twitter_train_data1.reset_index(drop= True)
twitter_train_data1.sentence = twitter_train_data1.sentence.apply(lambda x : re.sub('[0-9]+', '', str(x)).split())
Counter(np.array(twitter_train_data1['label']))

In [0]:
#build Vocabulary
count = []
for i in range(len(twitter_train_data1)):
  text = twitter_train_data1.sentence[i]
  for word in text:
    count.append(word)
vocab = list(OrderedDict.fromkeys(sorted(count )))

def sentence_to_int(text):
  convert_int = []
  convert_int.append([vocab_to_int[word] for word in text])
  return convert_int


vocab_to_int = {word: ii for ii, word in enumerate(vocab,1)}

twitter_train_data1.sentence = twitter_train_data1.sentence.apply(lambda x:  list(chain.from_iterable(sentence_to_int(x))))
#twitter_train_data1.sentence = twitter_train_data1.sentence.apply(lambda x: list(chain.from_iterable(x)))

In [0]:
#convert text to integers 
max_length = []
remove = []
for i in range(len(twitter_train_data1.sentence)):
  each_element =len(twitter_train_data1.sentence[i])
  max_length.append(each_element)
  if each_element ==0:
    remove.append(i)
max(max_length)

twitter_train_data1 =twitter_train_data1.drop(remove, axis=0).reset_index(drop= True)
twitter_train_data1.head(10)

In [0]:
#copy the data set and work on the copied dataset
twitter_train_data_2 = twitter_train_data1.copy(deep= True)
twitter_train_data_3 = twitter_train_data1.copy(deep= True)
 

In [0]:
#pad input such tht each sentence has equal length

#pad sequence -1
def pad_features(x,seq_length):
  x = x[:seq_length]+[0]*(seq_length - len(x))
  return x
  
twitter_train_data_2.sentence = twitter_train_data_2.sentence.apply(lambda x: pad_features(x, 32))


#pad sequence -1
def trimmer(x, size, filler=0):
   return list(islice(chain(x, repeat(filler)), size))

twitter_train_data_3.sentence = twitter_train_data_3.sentence.apply(lambda x: trimmer(x, 30, filler=0))
twitter_train_data_3.sentence = twitter_train_data_3.sentence.apply(lambda x:  np.array((x)))



#reshape data
def reshape_sentence(x):
  return np.asarray(x).reshape(1,30)

def reshape_label(x):
  return np.asarray(x).reshape(1,1)



In [0]:
#seperate data into train and test 

def data_extract_train(x, length):
  train_x = np.zeros((1,30))
  for i in range(length):
    train_x = np.append(train_x,np.array(x[i]).reshape(1,30) ,axis = 0)
  return train_x[1:,:]

def data_extract_train_label(x, length):
  train_y = np.zeros((1))
  for i in range(length):
    train_y = np.append(train_y,np.array(x[i]).reshape(-1) ,axis = 0)
  return train_y[1:]

split_frac = 0.8
split_idx = int(len(twitter_train_data_3)*split_frac)
Training_data , remainging_data = twitter_train_data_3.iloc[:split_idx,:], twitter_train_data_3.iloc[split_idx:,:]

test_idx = int(len(remainging_data)*0.5)
Validation_data = remainging_data.iloc[:test_idx,:].reset_index(drop= True)
Testing_data = remainging_data.iloc[test_idx:,:].reset_index(drop= True)


train_x = Training_data['sentence'].to_numpy()
train_y = Training_data['label'].to_numpy()
train_x = np.array(list(chain.from_iterable(train_x))).reshape(-1,30)


valid_x = Validation_data['sentence'].to_numpy()
valid_y = Validation_data['label'].to_numpy()
valid_x = np.array(list(chain.from_iterable(valid_x))).reshape(-1,30)

test_x = Testing_data['sentence'].to_numpy() 
test_y = Testing_data['label'].to_numpy()
test_x = np.array(list(chain.from_iterable(test_x))).reshape(-1,30)

#Model 1: Simple LSTM

In [0]:
#creating tensor dataset
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

#dataloaders
batch_size = 50

#shuffle training data
train_loader = DataLoader(train_data ,batch_size= batch_size , shuffle = True , drop_last=True)
valid_loader  = DataLoader(valid_data ,batch_size= batch_size , shuffle = True, drop_last=True )
test_loader = DataLoader(test_data ,batch_size= batch_size , shuffle = True, drop_last=True )


#obtain a batch of training
data_iter = iter(train_loader)
sample_x , sample_y = data_iter.next()

In [0]:
import torch.nn as nn 

class SentimentRNN(nn.Module):
  """
  LSTM modeul used for sentiment analysis

  """

  def __init__(self, vocab_size, ouput_size , embedding_dim ,hidden_dim, n_layer , drop_prob = 0.5):
    """
    initalize the model by setting up the layer

    """
    super(SentimentRNN, self).__init__()

    self.output_size = output_size
    self.n_layer = n_layer
    self.hidden_dim = hidden_dim

    #embedding and LSTM layers
    self.embedding = nn.Embedding(vocab_size , embedding_dim)
    self.lstm = nn.LSTM(embedding_dim , hidden_dim , n_layer , dropout = drop_prob , batch_first = True)

    #dropout layer
    self.dropout = nn.Dropout(0.3)

    #liner and Sigmoid layer
    self.fc = nn.Linear(hidden_dim , output_size)
    self.sig = nn.Sigmoid()


  def forward(self , x, hidden):
    """
    Perform a forward pass of the model on some input and hidden state

    """
    batch_size = x.size(0)

    #embedding and lstm_out
    x = x.long()
    embeds = self.embedding(x)
    lstm_out , hidden = self.lstm(embeds , hidden)


    #stack up lstm outputs
    lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)


    #dropout and fully connected layer 

    out = self.dropout(lstm_out)
    out = self.fc(out)

    #sigmoid function
    sig_out = self.sig(out)

    #reshape to be batch_size first
    sig_out = sig_out.view(batch_size , -1)
    sig_out = sig_out[:,-1] #get last batch label

    #return last sigmoid output and the hidden state
    return sig_out , hidden 


  def init_hidden(self, batch_size):
    """
    intialize hidden state
    """
    weight = next(self.parameters()).data

    hidden = (weight.new(self.n_layer , batch_size , self.hidden_dim).zero_(),
              weight.new(self.n_layer, batch_size , self.hidden_dim).zero_())
    
    return hidden


In [0]:
vocab_size = len(vocab_to_int) +1
output_size = 1
embedding_dim = 200
hidden_dim = 512
n_layer = 2

net = SentimentRNN(vocab_size,output_size, embedding_dim, hidden_dim, n_layer, drop_prob=0.5)
print(net)


#triaining
lr = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = lr)

In [0]:
epochs = 10

counter = 0

print_every = 100

clip = 5 #gradient clipping

net.train()

for e in range(epochs):
  #initialize the hidden state
  h = net.init_hidden(batch_size)

  #batch loop

  for inputs ,labels in train_loader:
    counter += 1



    #create new variables for the hidden state, otherwize
    #wed backpropagate through the entire training history

    h = tuple([each.data for each in h])

    #zero accumalute gradients
    net.zero_grad()

    #get output from the model
    output, h = net(inputs, h)

    #calculate the loss  and perform backprop
    loss = criterion(output.squeeze(), labels.float())
    loss.backward()
    

    #clip_grad_norm helps prvent the exploding gradient problem in RNN
    nn.utils.clip_grad_norm(net.parameters(),clip)
    optimizer.step()

    #loss stats
    if counter % print_every ==0:
      #get validation loss-
      val_h = net.init_hidden(batch_size)

      val_losses = []
      net.eval()

      for inputs ,labels in valid_loader:

        val_h = tuple([each.data for each in val_h])

        ouput, val_h = net(inputs, val_h)
        val_loss = criterion(output.squeeze(), labels.float())

        val_losses.append(val_loss.item())


      net.train()
      print("Epoch:{}/{}".format(e+1,epochs),
            "Steps: {}...".format(counter),
            "Loss: {:.6f}".format(loss.item()),
            "Val loss: {:.6f}".format(np.mean(val_losses)))


In [0]:
test_losses = []
num_correct = 0

#init hidden state
h= net.init_hidden(batch_size)

net.eval()

for inputs , labels in test_loader:

  #creating new variable for the hidden state, otherwise
  #we'd backdrop through the entire training history

  h = tuple([each.data for each in h])

  #get predicted outputs
  output , h = net(inputs,h)

  #calulate loss
  test_loss = criterion(output.squeeze() , labels.float())
  test_losses.append(test_loss.item())

  #convert output probabilities to predict 0 or 1
  pred = torch.round(output.squeeze()) # rounds to the nearest integer

  #compare
  correct_tensor = pred.eq(labels.float().view_as(pred))

  correct = np.squeeze(correct_tensor.numpy())
  num_correct += np.sum(correct)


print('Test loss: {:.3f}'.format(np.mean(test_losses)))


#accuracy over all test
test_acc = num_correct/ len(test_loader.dataset)
print('Test accuracy : {:.3f}'.format(test_acc))



#Model 2: Bi- -LSTM

In [0]:
#reshape the data for this and further models
train_y = train_y.reshape(-1,1)
valid_y = valid_y.reshape(-1,1)
test_y = test_y.reshape(-1,1) 

max_features = len(vocab_to_int)+5
max_length = 30

def generator(X_data, y_data, batch_size):

  samples_per_epoch = X_data.shape[0]
  number_of_batches = samples_per_epoch/batch_size
  counter=0

  while 1:

    X_batch = np.array(X_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
    y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
    counter += 1
    yield X_batch,y_batch

    #restart counter to yeild data in the next epoch as well
    if counter >= number_of_batches:
        counter = 0


In [0]:
rms2 = RMSprop()
#model
bilstm_model = Sequential()
bilstm_model.add(Embedding(max_features , 200, input_length= max_length))
bilstm_model.add(Bidirectional(LSTM(512, return_sequences=True)))
bilstm_model.add(Dropout(0.7))
bilstm_model.add(Bidirectional(LSTM(128)))
bilstm_model.add(Dropout(0.7))
bilstm_model.add(Dense(1, activation='sigmoid'))
bilstm_model.compile( optimizer=rms2 ,loss='binary_crossentropy', metrics=['accuracy'])
print(bilstm_model.summary())
bilstm_model.fit_generator(generator(train_x,train_y,1000), steps_per_epoch=20 ,epochs=10, verbose=1, callbacks=None, validation_data=generator(valid_x,valid_y,1000), validation_steps=5, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)


In [0]:
#prediction
y_pred_bilstm_prob = bilstm_model.predict(test_x)
y_pred_bilstm_classes = bilstm_model.predict_classes(test_x)


#prediction Statistics
score_bilstm = bilstm_model.evaluate(test_x, test_y, verbose = 1)
bilstm_accuracy = accuracy_score(test_y, y_pred_bilstm_classes)
bilstm_precision = precision_score(test_y, y_pred_bilstm_classes)
rocAuc_bilstm = roc_auc_score(test_y, y_pred_bilstm_classes)
falsePositiveRate, truePositiveRate, _ = roc_curve(test_y,y_pred_bilstm_prob)


#ROC curve
plt.figure()
plt.plot(falsePositiveRate, truePositiveRate, color='green',
         lw=3, label='ROC curve (area = %0.2f)' % rocAuc_bilstm)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of Sentiment Analysis using BiLSTM model')
plt.legend(loc="lower right")
plt.show()


#accuracy and precsion
print(bilstm_accuracy) 
print(bilstm_precision)

#Model 3:  CNN-LSTM

In [0]:
tf.reset_default_graph()
#model
cnn_lstm = Sequential()
cnn_lstm.add(Embedding(max_features , 200, input_length= max_length))
cnn_lstm.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='causal'))
cnn_lstm.add(MaxPooling1D(pool_size=2))
cnn_lstm.add(Dropout(0.7))
cnn_lstm.add(LSTM(units=256))
cnn_lstm.add(Dropout(0.7))
cnn_lstm.add(Dense(1, activation='sigmoid'))
cnn_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(cnn_lstm.summary())


# model training
cnn_lstm.fit_generator(generator(train_x,train_y,1000), steps_per_epoch=50 ,epochs=10, verbose=1, callbacks=None, validation_data=generator(valid_x,valid_y,1000), validation_steps=3, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

In [0]:
#prediction
y_predict_cnn_lstm_prob = cnn_lstm.predict(test_x)
y_pred_cnn_lstm_classes = cnn_lstm.predict_classes(test_x)

#analyze the results
#prediction Statistics
score_cnn_lstm = cnn_lstm.evaluate(test_x, test_y, verbose = 1)
cnn_lstm_accuracy = accuracy_score(test_y, y_pred_cnn_lstm_classes)
cnn_lstm_precision = precision_score(test_y, y_pred_cnn_lstm_classes)
rocAuc_cnn_lstm= roc_auc_score(test_y, y_pred_cnn_lstm_classes)
falsePositiveRate, truePositiveRate, _ = roc_curve(test_y, y_predict_cnn_lstm_prob)

#ROC AUC curve
plt.figure()

plt.plot(falsePositiveRate, truePositiveRate, color='green',
         lw=3, label='ROC curve (area = %0.2f)' % rocAuc_cnn_lstm)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of Sentiment Analysis using CNN-LSTM model')
plt.legend(loc="lower right")
plt.show()


#accuracy and precsion
print(cnn_lstm_accuracy )
print(cnn_lstm_precision)

#Model 4: Bi-LSTM with Attention

In [0]:
tf.reset_default_graph()

#attenstion Layer
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights


Bilstm_attn_input = tf.keras.layers.Input(shape=(max_length,), dtype='int32')

embedded_sequences = tf.keras.layers.Embedding(max_features, 200, input_length=max_length)(Bilstm_attn_input)
lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional \
    (tf.keras.layers.LSTM
     (256,
      dropout=0.2,
      return_sequences=True,
      return_state=True,
      recurrent_activation='relu',
      recurrent_initializer='glorot_uniform'))(embedded_sequences)#(lstm)

state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])


context_vector, attention_weights = Attention(30)(lstm, state_h)
output = keras.layers.Dense(1, activation='sigmoid')(context_vector)
bilstm_attn_model = keras.Model(inputs=Bilstm_attn_input, outputs=output)

# summarize layers
print(bilstm_attn_model.summary())
rms = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
bilstm_attn_model.compile(loss='binary_crossentropy', optimizer=rms , metrics=['accuracy'])

In [0]:
bilstm_attn_model.fit_generator(generator(train_x,train_y,1000),steps_per_epoch= 30,epochs=15, verbose=1, callbacks=None, validation_data=generator(valid_x,valid_y,1000), validation_steps=5, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

In [0]:
#prediction
y_predict_bilstm_attn_prob = bilstm_attn_model.predict(test_x)
y_predict_bilstm_attn_classes = (y_predict_bilstm_attn_prob> 0.5).astype(np.int)

#analyze the results
#prediction Statistics
score_bilstm_attn = bilstm_attn_model.evaluate(test_x, test_y, verbose = 1)
bilstm_attn_accuracy = accuracy_score(test_y, y_predict_bilstm_attn_classes)
bilstm_attn_precision = precision_score(test_y, y_predict_bilstm_attn_classes)
rocAuc_bilstm_attn = roc_auc_score(test_y, y_predict_bilstm_attn_classes)
falsePositiveRate, truePositiveRate, _ = roc_curve(test_y, y_predict_bilstm_attn_prob)

#ROC AUC curve
plt.figure()

plt.plot(falsePositiveRate, truePositiveRate, color='green',
         lw=3, label='ROC curve (area = %0.2f)' % rocAuc_bilstm_attn)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of Sentiment Analysis using Bi-LSTM_attn model')
plt.legend(loc="lower right")
plt.show()


#accuracy and precsion
print(bilstm_attn_accuracy )
print(bilstm_attn_precision)